<a href="https://colab.research.google.com/github/jfmaramburo/BEDU_COVID19/blob/main/COVID19_leerCSV_guardarMYSQL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Declaración de Imports
import pandas as pd
import sqlalchemy
import yaml 

In [ ]:
#Leer informacion desde archivo CSV
df = pd.read_csv("210704COVID19MEXICO.csv")

In [ ]:
#Revisamos total de Registros y Tipos de Datos
total_registros = len(df.index)
print('Total de Regitros Leidos:',total_registros)
df.dtypes

In [ ]:
#Validamos que no se encuentre ningun registro NA
df.isna().sum()

In [ ]:
#Diccionario de Columnas que no ocuparemos
col_borrar = ['ORIGEN'
             ,'ENTIDAD_UM'
             ,'ENTIDAD_NAC'
             ,'TIPO_PACIENTE'
             ,'FECHA_SINTOMAS'
             ,'FECHA_DEF'
             ,'INTUBADO'
             ,'NEUMONIA'
             ,'EDAD'
             ,'NACIONALIDAD'             
             ,'EMBARAZO'                 
             ,'HABLA_LENGUA_INDIG'       
             ,'INDIGENA'                 
             ,'DIABETES'                 
             ,'EPOC'                     
             ,'ASMA'                     
             ,'INMUSUPR'                 
             ,'HIPERTENSION'             
             ,'OTRA_COM'                 
             ,'CARDIOVASCULAR'           
             ,'OBESIDAD'                 
             ,'RENAL_CRONICA'            
             ,'TABAQUISMO'               
             ,'OTRO_CASO'                
             ,'TOMA_MUESTRA_LAB'         
             ,'TOMA_MUESTRA_ANTIGENO'    
             ,'RESULTADO_ANTIGENO'    
             ,'MIGRANTE'              
             ,'PAIS_ORIGEN'
              ,'UCI']

#Pasar a nuevo data frame informacion con eliminado de columnas
df_mexico = df.drop(columns=col_borrar)

In [ ]:
#convertimos los tipos de datos a fecha y validamos tipos de Datos:
diccionario_de_conversion = {
    'FECHA_ACTUALIZACION' : 'datetime64[ns]',
    'FECHA_INGRESO' : 'datetime64[ns]'
}

df_mexico = df_mexico.astype(diccionario_de_conversion)
df_mexico.dtypes

In [ ]:
#Realizamos Filtrado por Clasificacion y Nacionalidad = México:
# 1: CASO DE COVID-19 CONFIRMADO POR ASOCIACIÓN CLÍNICA EPIDEMIOLÓGICA
# 2: CASO DE COVID-19 CONFIRMADO POR COMITÉ DE  DICTAMINACIÓN
# 3: CASO DE SARS-COV-2  CONFIRMADO
# 6: CASO SOSPECHOSO

clasificaciones = [1,2,3,6]
df_mexico = df_mexico[df_mexico.CLASIFICACION_FINAL.isin (clasificaciones)& (df_mexico.PAIS_NACIONALIDAD == 'México') ]

total_mexico = len(df_mexico.index)
print('Total de Casos en México:',total_mexico)

In [ ]:
#Obtener DataFrame por Fecha de Ingreso, Entidad Federativa y tipo de Sexo
df_res_mexico = df_mexico.groupby(['FECHA_INGRESO', 'ENTIDAD_RES','SEXO'] ).size().reset_index(name='CASOS')

In [ ]:
df_res_mexico

In [ ]:
#Obtener datos de conexion
with open("config.yaml", "r") as f:
     params = yaml.load(f, Loader=yaml.FullLoader)
    
#Guadardar DataFrame DF_RES_MEXICO en Mysql
database_username = params['username']
database_password = params['password2']
database_ip       = params['host']
database_name     = params['database']
database_connection = sqlalchemy.create_engine('mysql+mysqlconnector://{0}:{1}@{2}/{3}'.
                                               format(database_username, database_password, 
                                                      database_ip, database_name))

df_res_mexico.to_sql(con=database_connection, name='casos_global', if_exists='replace')